In [30]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


<IPython.core.display.Javascript object>

Step: Load data

In [5]:
import pandas as pd
import os
from pyprojroot import here
import numpy as np
from skimpy import clean_columns
path_data = here("./data")
os.chdir(path_data)
data_iu = pd.read_csv("iu_data.csv")
data_iu=  clean_columns(data_iu)



17 column names have been cleaned

<IPython.core.display.Javascript object>

In [19]:
import pandas as pd

# create a DataFrame with a column containing NaNs
df = pd.DataFrame({"A": [1, 2, np.nan, 3]})

# one-hot encode the 'A' column and keep NaNs as NaNs
encoded_df = pd.get_dummies(df["A"], dummy_na=True)

# concatenate the encoded DataFrame with the original DataFrame
df = pd.concat([df, encoded_df], axis=1)

print(df)

     A  1.0  2.0  3.0  NaN
0  1.0    1    0    0    0
1  2.0    0    1    0    0
2  NaN    0    0    0    1
3  3.0    0    0    1    0


<IPython.core.display.Javascript object>

In [20]:
data_iu["ncaa_tournament"].value_counts(dropna=False)

data_ncaa_tournament = pd.get_dummies(data_iu["ncaa_tournament"], dummy_na=True)
data_ncaa_tournament = clean_columns(data_ncaa_tournament)
data_ncaa_tournament = data_ncaa_tournament.rename(columns={"nan": "nan_tournament"})

7 column names have been cleaned

<IPython.core.display.Javascript object>

In [21]:
data_iu_encoded = pd.concat(
    [data_iu.drop(columns="ncaa_tournament"), data_ncaa_tournament], axis=1
)

data_iu_encoded

season   w  ... won_ncaa_tournament_national_final  nan_tournament
0    2022-23  23  ...                                  0               0
1    2021-22  21  ...                                  0               0
2    2020-21  12  ...                                  0               1
3    2019-20  20  ...                                  0               1
4    2018-19  19  ...                                  0               1
..       ...  ..  ...                                ...             ...
118  1904-05   5  ...                                  0               1
119  1903-04   5  ...                                  0               1
120  1902-03   8  ...                                  0               1
121  1901-02   4  ...                                  0               1
122  1900-01   1  ...                                  0               1

[123 rows x 24 columns]

<IPython.core.display.Javascript object>

Step: Limit data to 1939 and later and include a year var

In [31]:
data_iu_encoded["coach_es"] = data_iu_encoded["coach_es"].str.replace(r"\s*\(.*\)", "")

<IPython.core.display.Javascript object>

In [52]:
import pandas as pd
import numpy as np


def filter_and_encode_categorical_data(data, column_name, percentile_threshold):
    """
    This function takes a pandas DataFrame, a column name, and a percentile threshold as inputs.
    It then encodes the categorical data in the specified column using pd.get_dummies, keeps only
    the response options with a frequency above the specified percentile threshold, and returns
    NaNs as NaNs.

    :param data: pandas.DataFrame, the input data.
    :param column_name: str, the name of the column containing the categorical data.
    :param percentile_threshold: float, the percentile threshold to filter response options.
    :return: pandas.DataFrame, the transformed data with the specified column encoded.
    """

    # Create a boolean mask that flags NaNs in the column
    nan_mask = data[column_name].isna()

    # Calculate the frequency of each category in the column
    category_counts = data[column_name].value_counts()

    # Calculate the threshold count based on the percentile
    threshold_count = np.percentile(category_counts, percentile_threshold)

    # Filter the categories with a count above the threshold
    filtered_categories = category_counts[category_counts > threshold_count].index

    # Create a new column with filtered categories, and replace less frequent categories with NaN
    filtered_data = data[column_name].apply(
        lambda x: x if x in filtered_categories else np.nan
    )

    # Encode the categorical data in the column with pd.get_dummies
    encoded_data = pd.get_dummies(filtered_data, dummy_na=False)

    # Reintroduce NaNs in the encoded data using the boolean mask
    for col in encoded_data.columns:
        encoded_data.loc[nan_mask, col] = np.nan

    # Combine the original data with the encoded columns, dropping the original column
    transformed_data = data.drop(column_name, axis=1).join(encoded_data)

    return transformed_data

<IPython.core.display.Javascript object>

In [65]:
def fun_hot_encode_limit(data, column_name, threshold=0.01):
    encoded_data = pd.get_dummies(data_iu_encoded["coach_es"], dummy_na=True)
    threshold = 0.05
    data_keep = (
        pd.DataFrame(
            data_iu_encoded["coach_es"].value_counts(normalize=True, dropna=False)
        )
        .reset_index()
        .query("coach_es >= @threshold")
    )
    encoded_data = clean_columns(encoded_data.filter(items=data_keep["index"]))
    return encoded_data

<IPython.core.display.Javascript object>

In [66]:
fun_hot_encode_limit(data=data_iu_encoded, column_name="coach_es", threshold=0.01)

4 column names have been cleaned

bob_knight  branch_mc_cracken  everett_dean  tom_crean
0             0                  0             0          0
1             0                  0             0          0
2             0                  0             0          0
3             0                  0             0          0
4             0                  0             0          0
..          ...                ...           ...        ...
118           0                  0             0          0
119           0                  0             0          0
120           0                  0             0          0
121           0                  0             0          0
122           0                  0             0          0

[123 rows x 4 columns]

<IPython.core.display.Javascript object>

index  coach_es
0        Bob Knight  0.235772
1  Branch McCracken  0.195122
2      Everett Dean  0.113821
3         Tom Crean  0.073171

<IPython.core.display.Javascript object>

Bob Knight  Branch McCracken  Everett Dean  Tom Crean
0             0                 0             0          0
1             0                 0             0          0
2             0                 0             0          0
3             0                 0             0          0
4             0                 0             0          0
..          ...               ...           ...        ...
118           0                 0             0          0
119           0                 0             0          0
120           0                 0             0          0
121           0                 0             0          0
122           0                 0             0          0

[123 rows x 4 columns]

<IPython.core.display.Javascript object>

In [54]:
filter_and_encode_categorical_data(
    data=data_iu_encoded, column_name="coach_es", percentile_threshold=0.5
)

season   w   l  ...  Tom Crean Willis Coval Zora Clevenger
0    2022-23  23  12  ...        0.0          0.0            0.0
1    2021-22  21  14  ...        0.0          0.0            0.0
2    2020-21  12  15  ...        0.0          0.0            0.0
3    2019-20  20  12  ...        0.0          0.0            0.0
4    2018-19  19  16  ...        0.0          0.0            0.0
..       ...  ..  ..  ...        ...          ...            ...
118  1904-05   5  12  ...        0.0          0.0            1.0
119  1903-04   5   4  ...        0.0          1.0            0.0
120  1902-03   8   4  ...        0.0          1.0            0.0
121  1901-02   4   4  ...        0.0          0.0            0.0
122  1900-01   1   4  ...        0.0          0.0            0.0

[123 rows x 38 columns]

<IPython.core.display.Javascript object>

Step: Make season a year